In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
#Data prep
shot_df = pd.read_csv("Hackathon_sv_shot_summary_2015-16.txt", sep=" ")

#have 165 occurences of null for closet defender id
shot_df_clean = shot_df.replace({'(null)': -1})

#shot_df = shot_df.drop(shot_df.index[222316])
shot_df_clean = shot_df_clean[shot_df_clean['SEASON'] != "GAME_ID"]

#convert columns to numbers
shot_df_clean[["SEASON", "GAME_ID", "SV_GAME_ID", "TEAM_ID", "SV_TEAM_ID", "PERSON_ID", "SV_PLAYER_ID", "PERIOD", "GAME_CLOCK", "WALL_CLOCK", "DRIBBLES", "SHOT_DIST", "TOUCH_TIME", "CLOSE_DEF_PERSON_ID", "CLOSE_DEF_SV_PLAYER_ID", "CLOSE_DEF_DIST", "PT_VALUE", "PTS"]] = shot_df_clean[["SEASON", "GAME_ID", "SV_GAME_ID", "TEAM_ID", "SV_TEAM_ID", "PERSON_ID", "SV_PLAYER_ID", "PERIOD", "GAME_CLOCK", "WALL_CLOCK", "DRIBBLES", "SHOT_DIST", "TOUCH_TIME", "CLOSE_DEF_PERSON_ID", "CLOSE_DEF_SV_PLAYER_ID", "CLOSE_DEF_DIST", "PT_VALUE", "PTS"]].apply(pd.to_numeric)

#convert missed/made to 0/1
shot_df_clean['SHOT_RESULT'] = shot_df_clean['SHOT_RESULT'].replace({'missed': 0})
shot_df_clean['SHOT_RESULT'] = shot_df_clean['SHOT_RESULT'].replace({'made': 1})

#classify shot distances
shot_df_clean.loc[shot_df_clean['PT_VALUE'] == 3, 'SHOT_TYPE'] = "Three Pointer"
shot_df_clean.loc[(shot_df_clean['SHOT_DIST'] >= 15) & (shot_df_clean['PT_VALUE'] == 2), 'SHOT_TYPE'] = "Long Two-Pointer"
shot_df_clean.loc[(shot_df_clean['SHOT_DIST'] >= 5) & (shot_df_clean['SHOT_DIST'] < 15), 'SHOT_TYPE'] = "Midrange"
shot_df_clean.loc[shot_df_clean['SHOT_DIST']< 5, 'SHOT_TYPE'] = "Layup"



/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,3,5,6,8,9,11,12,13,14,15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
shot_df_data = shot_df_clean[['SHOT_RESULT', "SHOT_DIST", "DRIBBLES", "CLOSE_DEF_DIST", "TOUCH_TIME", "SV_PLAYER_ID", "CLOSE_DEF_SV_PLAYER_ID"]]


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(shot_df_data.drop('SHOT_RESULT',axis=1), 
                                                    shot_df_data['SHOT_RESULT'], test_size=0.3, 
                                                    random_state=101)

In [6]:
from sklearn.tree import DecisionTreeClassifier

In [72]:
dtree = DecisionTreeClassifier(min_samples_leaf = 50)
dtree.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [73]:
predictions = dtree.predict(X_test)

In [74]:
from sklearn.metrics import classification_report,confusion_matrix

In [75]:
print(classification_report(Y_test,predictions))

             precision    recall  f1-score   support

          0       0.61      0.70      0.65     36569
          1       0.56      0.45      0.50     30127

avg / total       0.59      0.59      0.58     66696



In [76]:
print(confusion_matrix(Y_test,predictions))

[[25621 10948]
 [16431 13696]]


In [77]:
predictions = dtree.predict_proba(X_test)
shot_probability = []
for i in predictions:
    shot_probability.append(round(i[1], 4))


In [78]:
prob_df = X_test.copy()
prob_df['SHOT_PROBABILITY'] = shot_probability

In [79]:
prob_df

,SHOT_DIST,DRIBBLES,CLOSE_DEF_DIST,TOUCH_TIME,SV_PLAYER_ID,CLOSE_DEF_SV_PLAYER_ID,SHOT_PROBABILITY
69447,13.44,0,2.00,0.68,4244,3930,0.1481
179348,1.38,4,1.39,4.50,4724,5336,0.7193
136407,2.99,0,2.28,0.47,4477,5355,0.7759
186549,21.75,1,6.44,1.60,5318,4718,0.4304
38447,11.26,0,3.54,0.89,4489,4130,0.4615
57420,7.00,1,1.99,2.00,5347,3653,0.4571
179978,12.52,5,3.84,5.64,3607,4651,0.2405
108429,3.07,0,4.60,0.60,4920,4244,0.7636
150019,0.66,0,0.71,0.04,4309,4300,0.4386
27640,15.43,2,2.18,3.70,5464,5155,0.2361


In [80]:
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.metrics import brier_score_loss

In [81]:
probs = dtree.predict_proba(X_test)
# keep the predictions for class 1 only
                               
probs = probs[:, 1]

In [82]:
loss = log_loss(Y_test, probs)
loss
#terrible

0.6942301269270305

In [83]:
auc_loss = roc_auc_score(Y_test, probs)
auc_loss

0.6102035864266558

In [84]:
brier_loss = brier_score_loss(Y_test, probs)
brier_loss

0.24357981347860244

Random Forest

In [85]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [88]:
rfc_pred = rfc.predict(X_test)
predictions = dtree.predict(X_test)

In [89]:
print(classification_report(Y_test,predictions))

             precision    recall  f1-score   support

          0       0.61      0.70      0.65     36569
          1       0.56      0.45      0.50     30127

avg / total       0.59      0.59      0.58     66696



In [90]:
print(confusion_matrix(Y_test,predictions))

[[25621 10948]
 [16431 13696]]


In [92]:
predictions = rfc.predict_proba(X_test)
shot_probability_rfc = []
for i in predictions:
    shot_probability_rfc.append(round(i[1], 4))

In [93]:
prob_df_rfc = X_test.copy()
prob_df_rfc['SHOT_PROBABILITY'] = shot_probability_rfc

In [94]:
prob_df_rfc

,SHOT_DIST,DRIBBLES,CLOSE_DEF_DIST,TOUCH_TIME,SV_PLAYER_ID,CLOSE_DEF_SV_PLAYER_ID,SHOT_PROBABILITY
69447,13.44,0,2.00,0.68,4244,3930,0.290
179348,1.38,4,1.39,4.50,4724,5336,0.565
136407,2.99,0,2.28,0.47,4477,5355,0.700
186549,21.75,1,6.44,1.60,5318,4718,0.280
38447,11.26,0,3.54,0.89,4489,4130,0.520
57420,7.00,1,1.99,2.00,5347,3653,0.325
179978,12.52,5,3.84,5.64,3607,4651,0.335
108429,3.07,0,4.60,0.60,4920,4244,0.850
150019,0.66,0,0.71,0.04,4309,4300,0.235
27640,15.43,2,2.18,3.70,5464,5155,0.185


In [95]:
probs = rfc.predict_proba(X_test)
# keep the predictions for class 1 only
                               
probs = probs[:, 1]

In [97]:
loss = log_loss(Y_test, probs)
loss

0.6610525408562202

In [98]:
auc_loss = roc_auc_score(Y_test, probs)
auc_loss

0.6227408372038115

In [99]:
brier_loss = brier_score_loss(Y_test, probs)
brier_loss

0.2344786766596489